# Getting the Model

First we install the requirements as mentioned in the README.md file.

In [4]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 15.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 18.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.7 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 6.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 15.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pytz
    Found existing installation: pytz 2023.3.post1
    Uninstalling pytz-2023.3.post1:
      Successfully uninstalled pytz-2023.3.post1
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling 

Import all the dependencies needed for the predict function.

In [6]:
import pandas as pd
import joblib
from pydantic import BaseModel, Field
from pydantic.tools import parse_obj_as

Create Pydantic models for data validation and modeling.

In [7]:
# Pydantic Models
class Student(BaseModel):
    student_id: str = Field(alias="Student ID")
    gender: str = Field(alias="Gender")
    age: str = Field(alias="Age")
    major: str = Field(alias="Major")
    gpa: str = Field(alias="GPA")
    extra_curricular: str = Field(alias="Extra Curricular")
    num_programming_languages: str = Field(alias="Num Programming Languages")
    num_past_internships: str = Field(alias="Num Past Internships")

    class Config:
        allow_population_by_field_name = True

class PredictionResult(BaseModel):
    good_employee: int

Create the predict function.

In [8]:
# Main Functionality
def predict(student):
    '''
    Returns a prediction on whether the student will be a good employee
    based on given parameters by using the ML model

    Parameters
    ----------
    student : dict
        A dictionary that contains all fields in Student
    
    Returns
    -------
    dict
        A dictionary satisfying type PredictionResult, contains a single field
        'good_employee' which is either 1 (will be a good employee) or 0 (will
        not be a good employee)
    '''
    # Use Pydantic to validate model fields exist
    student = parse_obj_as(Student, student)

    clf = joblib.load('./model.pkl')
    
    student = student.dict(by_alias=True)
    query = pd.DataFrame(student, index=[0])
    prediction = clf.predict(query) # TODO: Error handling ??

    return { 'good_employee': prediction[0] }

All the code above was provided in the draft pull request. Now we will use the sample from the README.md file to make sure it works as intended.

In [9]:
student = {
        "student_id": "student1",
        "major": "Computer Science",
        "age": "20",
        "gender": "M",
        "gpa": "4.0",
        "extra_curricular": "Men's Basketball",
        "num_programming_languages": "1",
        "num_past_internships": "2"
    }

In [10]:
predict(student)

{'good_employee': 1}

It works :) Now we can start evaluating the model.

# Evaluating Model Performance & Fairness

Since we don't know how the model provided in the draft pull request was trained, we want to evaluate it before implementing it into NodeBB. In the rest of the notebook, we will be evaluating the model's performance and fairness.

The engineer who created the model made the following assumptions:
- The model is used to predict whether a student will be a good candidate for a software engineering job.
- It is a binary classifier, where 1 means the student is a good candidate, and 0 means the student is not a good candidate.
- The model is trained on a dataset of students who have graduated from CMU, and have been working in the industry for at least 1 year.
- In order to prevent bias, they assumed that removing Gender (M, F) and Student ID from the dataset would be sufficient when it comes to training the model. They claimed that it is now Group Unaware, thus the model would be fair.

The model specification is as follows:

In [5]:
# X variable (input parameters)
# - Age (18 - 25)
# - Major (Computer Science, Information Systems, Business, Math,
#          Electrical and Computer Engineering, Statistics and Machine Learning)
# - GPA (0 - 4.0)
# - Extra Curricular Activities (Student Theatre, Buggy, Teaching Assistant, Student Government,
#     Society of Women Engineers, Women in CS, Volleyball, Sorority, Men's Basketball,
#     American Football, Men's Golf, Fraternity)
# - Number of Programming Languages (1, 2, 3, 4, 5)
# - Number of Past Internships (0, 1, 2, 3, 4)

# Y variable (output)
# - Good Candidate (0, 1)

The previous engineer has provided some examples on the usage of the model in the draft pull request.
You are provided with a test dataset, which contains a similar set of features and output (whether the student is a good candidate or not). This test dataset is a different set of students from the training dataset, and the evaluation of whether the student is a good candidate is done by a fair panel of recruiters, so it can be considered to be unbiased. Additionally, the panel of recruiters have provided you with additional context on the extracurricular activities in comments (marked with #).

Your test dataset is provided to you in the following format:

In [6]:
# X variable
# - Student ID
# - Gender (M, F)
# - Age (18 - 25)
# - Major (Computer Science, Information Systems, Business, Math,
#          Electrical and Computer Engineering, Statistics and Machine Learning)
# - GPA (0 - 4.0)
# - Extra Curricular Activities (Student Theatre, Buggy, Teaching Assistant, Student Government,
#     Society of Women Engineers, Women in CS, Volleyball, Sorority, Men's Basketball,
#     American Football, Men's Golf, Fraternity)
#   # Likely Co-Ed (Student Theatre, Buggy, Teaching Assistant, Student Government)
#   # Likely Majority Female (Society of Women Engineers, Women in CS, Volleyball, Sorority)
#   # Likely Majority Male (Men's Basketball, American Football, Men's Golf, Fraternity)
# - Number of Programming Languages (1, 2, 3, 4, 5)
# - Number of Past Internships (0, 1, 2, 3, 4)

# Y variable
# - Good Candidate (0, 1)

Before doing a thorough evaluation of the fairness of the model, you will start by doing preliminary analysis on the test dataset, and run the model on the test dataset to get the accuracy of the model. To do so, you will need to set up a Jupyter notebook to do this, you can either:

Use Google Colab to run the notebook in the cloud. (Recommended if you do not have experience with Jupyter notebooks)
Alternatively, set up a JupyterLab on your local machine. Additionally, you can use VSCode to run the notebook as well. (Recommended if you are experienced)
It is recommended that you use Python 3.9 or above when setting up the notebook.

After you have set up the notebook, you should:

Load the model and test dataset
Plot the distribution of the test dataset across all features (except Student ID) using any visualization library of your choice (e.g. pandas, matplotlib, seaborn, plotly, etc.). You should choose the appropriate visualization for each feature.
Predict the output of the test dataset using the model
Report the accuracy of the model, and the confusion matrix
Refer to the Resources & Documentation section if you need help with any of the above steps.

By the checkpoint deadline, your team will commit the Jupyter notebook to your repository, and submit a link to the Jupyter notebook with the basic analysis and usage of the model done.

In [7]:
# ⠀⢀⡤⢚⡭⣿⡟⠉⠉⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠉⠁⠀⠀⠀⠉⠉⠁⠀⠉⣿⠀⠀⠀⠀⠀
# ⠀⠀⠀⢁⢶⡟⠀⠀⠀⠘⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠸⠀⠀⠀⠀⡆⠁⢀⠀⡇⠠⣼⡄⠀⠀⠀⠀
# ⠀⠀⢠⠏⣾⠁⡄⠀⠀⡃⠀⢡⠀⠀⠀⠃⠀⠀⠀⠀⠀⠀⠀⡁⠀⠀⢠⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⣇⠇⠀⢀⡄⢸⢀⡆⢸⡰⡆⢸⠿⣧⠀⠀⠀⠀
# ⠀⠀⡜⣰⡏⠀⠧⠂⠰⠇⢰⣸⡄⠀⣼⡀⠀⠄⢠⠀⡀⠀⢠⡇⠀⢀⢾⠀⠀⢀⠇⠀⠀⠀⠀⠀⠀⡀⣸⠀⢀⢸⡟⠀⠀⡼⠀⡇⣼⡁⠆⡇⣧⢸⠀⠈⠀⠀⠀⠀
# ⠀⢠⣷⠇⠀⢸⢸⠀⢰⠄⣸⡟⡇⢀⠽⡇⢀⡇⠘⠰⠁⠀⡞⡇⠀⡜⢸⡀⠀⣸⡄⠀⢀⠀⠀⢀⠴⢃⡇⢀⣇⠞⣓⠲⣴⠁⢸⢰⡇⢧⠀⣸⢿⣼⠀⠀⠀⠀⠀⠀
# ⠀⣼⡏⢀⡆⠘⢸⠀⢸⠀⣿⣤⡇⣸⠀⡇⢸⡆⠂⢸⠀⢰⠇⡇⢰⠗⢻⡃⢸⣿⠀⠀⢸⠀⢀⡏⠀⣋⠅⣼⠋⣼⣿⡇⣇⠀⣿⠏⣧⢸⢠⣿⠀⠻⠀⠀⠀⠀⠀⠀
# ⢠⣿⢀⡞⠀⡀⢸⢀⠸⣾⣿⣹⣇⡇⣄⡇⣼⡇⠀⣼⠀⡾⠉⢱⡞⠀⢀⡇⡞⢹⠀⠀⣾⠀⣼⡇⢠⣿⢠⠇⠘⣿⡇⢣⣸⡄⡞⠀⣿⠈⣿⡏⠀⠀⠀⠀⠀⠀⠀⠀
# ⢸⣿⢻⠇⣠⡇⣹⢸⡆⣿⠿⣿⣿⣄⢸⡇⡏⢿⡮⢹⢰⡇⣠⣼⡷⣶⣿⣿⣁⢘⠠⠇⣿⢰⣱⡁⣸⣿⡼⠀⣾⠿⠏⢸⢋⣿⣷⣤⠙⣇⣿⣇⠀⠀⠀⠀⠀⠀⠀⠀
# ⠾⠁⠘⡰⢋⣧⣇⣸⣷⡞⠓⠺⢧⣘⣆⢻⠇⢸⡇⣸⣞⣹⡿⠛⠻⣏⣀⡭⠙⢛⡶⠦⣟⢫⣿⡿⠉⣿⠇⠀⡿⠁⠀⢧⣞⣿⡇⢻⣧⡘⣿⣿⡆⠀⠀⠀⠀⠀⠀⠀
# ⠀⢠⡿⠁⢸⠇⣼⠋⡿⣧⡀⠀⠀⠙⣿⡿⠀⢹⣶⠟⠋⠛⠻⠶⣤⣀⠀⠠⠔⠋⣣⣾⡏⡞⢸⠃⠀⡿⠀⠀⠁⠀⡰⠋⠘⠾⡇⠀⠙⠳⣿⣿⣿⡀⠀⠀⠀⠀⠀⠀
# ⠀⡾⠁⢀⡏⢰⠃⠀⡇⢿⠿⣿⣿⣿⣿⠷⠶⠾⢿⣦⣀⣀⣀⣀⣈⣿⣇⢀⣴⠟⠉⢸⡿⠁⠊⠀⠀⠃⣀⣄⣠⠞⠁⠀⠀⠀⠀⠀⠀⣰⣿⣿⣿⣷⣦⣀⠀⠀⠀⠀
# ⠈⠁⠀⠼⠁⠁⠀⠀⠃⢸⠀⢹⡉⠛⠋⠀⠀⠀⠀⠻⢿⣿⣿⣿⣿⣿⣿⣛⡟⠀⠀⠛⠃⠀⠀⠀⠀⢠⡿⠏⣠⡆⠀⠀⠀⠀⠀⣠⣾⣿⣿⣿⣿⣿⣿⣿⣿⣦⡀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⢷⣤⡀⠀⠀⠀⠀⠀⠀⠈⠙⠛⠛⠋⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⢾⠁⣰⣿⡇⠀⠀⠀⢀⣼⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣄
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠹⣿⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢟⣹⠀⠀⠀⣠⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢾⣯⣄⣀⡀⠀⠀⠀⠀⣀⡤⠖⠋⠀⠀⠀⠀⠀⢀⣠⡤⠀⠀⠀⢺⣿⡀⢀⣼⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠀⠀⢀⣩⡗⠒⠒⠉⠀⠀⠀⠀⠀⠀⠀⣠⣴⣿⠏⠀⠀⠀⠀⣾⠏⣰⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠧⠴⢿⡋⠁⠀⠀⠀⠀⠀⠀⠀⠀⣠⣞⢻⠟⠁⠀⠀⠀⠀⠀⣡⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢳⣄⡀⠀⠀⠀⠀⣀⣴⣮⣹⡿⠃⠀⠀⠀⠀⣀⣴⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿